In [1]:
#Programmer: Danish Khalid
import numpy as np
from urllib.request import *


training_images = 'https://raw.githubusercontent.com/rdanishk/Naive-Bayes-Classifier/master/trainingimages.txt'
training_labels = 'https://raw.githubusercontent.com/rdanishk/Naive-Bayes-Classifier/master/traininglabels.txt'

test_images = 'https://raw.githubusercontent.com/rdanishk/Naive-Bayes-Classifier/master/testimages.txt'
test_labels = 'https://raw.githubusercontent.com/rdanishk/Naive-Bayes-Classifier/master/testlabels.txt'

trainingImages = []
trainingLabels = []
testImages = []
testLabels = []

with urlopen(training_images) as response:
  for line in response:
    line = line.decode('utf-8').rstrip("\n\r")
    trainingImages.append(list(line))

with urlopen(training_labels) as response:
  for line in response:
    line = line.decode('utf-8').rstrip("\n\r")
    trainingLabels.append(list(line))

with urlopen(test_images) as response:
  for line in response:
    line = line.decode('utf-8').rstrip("\n\r")
    testImages.append(list(line))

with urlopen(test_labels) as response:
  for line in response:
    line = line.decode('utf-8').rstrip("\n\r")
    testLabels.append(list(line))

trainingImages = np.array(trainingImages)
trainingImages=np.reshape(trainingImages,(5000,28,28))
trainingImages[trainingImages==' '] = 0
trainingImages[trainingImages=='+'] = 1
trainingImages[trainingImages=='#'] = 1

trainingLabels = np.array(trainingLabels)

testImages = np.array(testImages)
testImages = np.reshape(testImages, (1000, 28, 28))
testImages[testImages == ' '] = 0
testImages[testImages == '+'] = 1
testImages[testImages == '#'] = 1

testLabels = np.array(testLabels)

uniqueLabels = [0,1,2,3,4,5,6,7,8,9]
digitIndexList = []
tempList = []

**Indexing all the Labels against the images:**

In [2]:
uniqueLabels = [0,1,2,3,4,5,6,7,8,9]
digitIndexList_Training = []
tempList = []

for i in uniqueLabels:
  for j in range(0,len(trainingLabels)):
    if int(trainingLabels[j][0]) == i:
      tempList.append(j)
  digitIndexList_Training.append(tempList)

digitIndexList_Testing = []
for i in uniqueLabels:
  for j in range(0, len(testLabels)):
    if int(testLabels[j][0]) == i:
      tempList.append(j)
    digitIndexList_Testing.append(tempList)


**Counting the number of 0's and 1's for each class:**

In [3]:
counter_zero = np.zeros((10,28,28))
counter_one = np.zeros((10,28,28))

for i in range(0, 5000):
  for j in range(0, 28):
    for k in range(0, 28):
      if int(trainingImages[i][j][k]) == 0:
        counter_zero[int(trainingLabels[i])][j][k] += 1
      else:
        counter_one[int(trainingLabels[i])][j][k] += 1

# print(counter_zero)
# print(counter_one)

**Class Probability:**

In [4]:
classProbability = [0 for _ in range(10)]
for i in range(0, 10):
  for j in range(0, len(trainingLabels)):
    if i == int(trainingLabels[j]):
      classProbability[i] += 1
  classProbability[i] /= 5000

print(classProbability)

[0.0958, 0.1126, 0.0976, 0.0986, 0.107, 0.0868, 0.1002, 0.11, 0.0924, 0.099]


**Calculating Priors:**

In [5]:
prior_zero = np.zeros((10,28,28))
prior_one = np.zeros((10,28,28))

list_labels = list(trainingLabels.astype(int))


k = 5

for i in range(10):
  for j in range(28):
    for k in range(28):
      prior_zero[i][j][k] = ( k + counter_zero[i][j][k] ) / ( (2 * k) + list_labels.count(i) )
      prior_one[i][j][k] = ( k + counter_one[i][j][k] ) / ( (2 * k) + list_labels.count(i) )

# print(prior_zero)
# print(prior_one)  


**Calculating Posteriors:**

In [6]:
import math
posterior = np.zeros((1000, 10))

for i in range(0, 1000):
  for p in range(0, 10):
    posterior[i][p] = math.log10(classProbability[p])
  for j in range(0, 28):
    for k in range(0, 28):
      if int(testImages[i][j][k]) == 0:
        for c in range(0, 10):
          posterior[i][c] += math.log10(prior_zero[c][j][k])
      else:
        for c in range(0, 10):
          posterior[i][c] += math.log10(prior_one[c][j][k])

print(posterior)


[[-160.67557842  -86.20880697 -116.58666851 ...  -73.22562031
   -97.13037181  -77.11991289]
 [ -86.861794   -180.07083656 -115.02742726 ... -132.23562691
  -109.13456581 -116.23585482]
 [-138.82193359  -96.94395326  -82.41942736 ... -148.2948769
   -83.89153137 -137.34112888]
 ...
 [-128.92010055 -110.98802848 -121.23662104 ... -128.7601555
   -94.0690503  -122.33440215]
 [-146.84510817 -126.52020657 -109.2990952  ...  -92.78296438
   -96.90321777  -65.99587983]
 [-128.61813181 -144.77254664 -130.07937005 ... -100.40738847
   -99.84310712  -87.54512217]]


**Predicting:**

In [7]:
prediction = []

for i in range(0,1000):
  prediction.append(np.where(posterior == np.amax(posterior[i]))[1].astype(int)[0])

print(prediction)

[7, 0, 2, 3, 1, 9, 7, 8, 1, 0, 4, 1, 9, 9, 4, 9, 2, 6, 8, 1, 3, 7, 9, 4, 8, 1, 8, 1, 3, 8, 1, 2, 8, 8, 0, 6, 2, 1, 1, 9, 1, 5, 3, 4, 8, 9, 5, 0, 9, 2, 2, 4, 9, 2, 1, 7, 2, 4, 9, 4, 4, 0, 9, 9, 2, 2, 3, 3, 9, 3, 5, 7, 7, 5, 8, 1, 2, 9, 6, 6, 4, 9, 3, 1, 0, 6, 9, 5, 9, 3, 9, 9, 1, 8, 0, 3, 9, 1, 3, 6, 7, 2, 5, 9, 7, 9, 6, 3, 6, 3, 7, 4, 6, 5, 8, 9, 9, 1, 8, 8, 8, 6, 7, 6, 8, 9, 7, 9, 7, 1, 9, 5, 2, 7, 3, 1, 1, 1, 2, 1, 4, 7, 4, 7, 3, 4, 5, 4, 0, 9, 3, 1, 9, 5, 0, 2, 7, 9, 4, 8, 4, 6, 4, 4, 9, 9, 3, 4, 5, 9, 8, 7, 3, 9, 2, 7, 5, 3, 4, 1, 1, 1, 6, 9, 1, 9, 7, 2, 0, 1, 5, 4, 4, 1, 9, 0, 6, 6, 4, 6, 5, 1, 5, 0, 9, 3, 8, 2, 9, 5, 2, 1, 6, 1, 1, 3, 7, 1, 0, 3, 0, 9, 4, 0, 6, 8, 2, 2, 3, 8, 4, 0, 9, 6, 5, 3, 1, 2, 1, 3, 1, 7, 9, 6, 7, 2, 0, 0, 3, 8, 1, 6, 3, 4, 1, 9, 3, 1, 5, 8, 1, 0, 2, 4, 4, 3, 4, 9, 8, 2, 9, 0, 4, 6, 4, 9, 7, 9, 3, 4, 1, 5, 9, 2, 3, 5, 8, 8, 8, 9, 3, 3, 6, 6, 0, 1, 6, 0, 3, 7, 4, 4, 1, 2, 9, 1, 4, 6, 4, 9, 7, 9, 8, 4, 4, 5, 1, 9, 1, 3, 1, 7, 9, 4, 8, 8, 1, 4, 9, 1, 9, 3, 2, 

**Accuracy:**

In [8]:
accuracy = 0
for i in range(0, 1000):
  if prediction[i] == int(testLabels[i]):
    accuracy += 1

accuracy = (accuracy / 1000) * 100


**Confusion Matrix:**

In [9]:
confusionMatrix = np.zeros((10, 10))

test_classes = testLabels.astype(int)
test_classes = test_classes.squeeze()
test_classes = test_classes.tolist()

for test, predict in zip(test_classes, prediction):
  confusionMatrix[test][predict] += 1

# print(confusionMatrix)

**Results:**

In [10]:
print("\n Confusion Matrix: \n")
print(confusionMatrix)

print("\n Accuracy: ", str(accuracy) + "%\n")

for i in range(0, 1000):
  print("Index: ", i, " Predicted Value: ", prediction[i], " Actual Value: ", test_classes[i] )




 Confusion Matrix: 

[[ 75.   0.   1.   0.   1.   5.   4.   0.   4.   0.]
 [  0. 105.   1.   0.   0.   1.   1.   0.   0.   0.]
 [  2.   6.  73.   6.   1.   0.   6.   2.   5.   2.]
 [  0.   2.   0.  80.   0.   3.   1.   6.   2.   6.]
 [  0.   2.   0.   0.  81.   0.   3.   0.   2.  19.]
 [  2.   2.   1.  17.   4.  52.   2.   2.   2.   8.]
 [  1.   9.   4.   0.   8.   4.  63.   0.   2.   0.]
 [  0.   7.   3.   0.   2.   0.   0.  79.   2.  13.]
 [  2.   4.   3.  13.   2.   5.   1.   2.  61.  10.]
 [  1.   2.   0.   3.  10.   2.   0.   2.   1.  79.]]

 Accuracy:  74.8%

Index:  0  Predicted Value:  7  Actual Value:  9
Index:  1  Predicted Value:  0  Actual Value:  0
Index:  2  Predicted Value:  2  Actual Value:  2
Index:  3  Predicted Value:  3  Actual Value:  5
Index:  4  Predicted Value:  1  Actual Value:  1
Index:  5  Predicted Value:  9  Actual Value:  9
Index:  6  Predicted Value:  7  Actual Value:  7
Index:  7  Predicted Value:  8  Actual Value:  8
Index:  8  Predicted Value:  1  Act

Index:  227  Predicted Value:  2  Actual Value:  2
Index:  228  Predicted Value:  3  Actual Value:  9
Index:  229  Predicted Value:  8  Actual Value:  8
Index:  230  Predicted Value:  4  Actual Value:  4
Index:  231  Predicted Value:  0  Actual Value:  0
Index:  232  Predicted Value:  9  Actual Value:  9
Index:  233  Predicted Value:  6  Actual Value:  5
Index:  234  Predicted Value:  5  Actual Value:  8
Index:  235  Predicted Value:  3  Actual Value:  5
Index:  236  Predicted Value:  1  Actual Value:  1
Index:  237  Predicted Value:  2  Actual Value:  2
Index:  238  Predicted Value:  1  Actual Value:  1
Index:  239  Predicted Value:  3  Actual Value:  3
Index:  240  Predicted Value:  1  Actual Value:  1
Index:  241  Predicted Value:  7  Actual Value:  7
Index:  242  Predicted Value:  9  Actual Value:  4
Index:  243  Predicted Value:  6  Actual Value:  5
Index:  244  Predicted Value:  7  Actual Value:  7
Index:  245  Predicted Value:  2  Actual Value:  2
Index:  246  Predicted Value:  

Index:  517  Predicted Value:  0  Actual Value:  0
Index:  518  Predicted Value:  2  Actual Value:  2
Index:  519  Predicted Value:  3  Actual Value:  3
Index:  520  Predicted Value:  7  Actual Value:  7
Index:  521  Predicted Value:  3  Actual Value:  5
Index:  522  Predicted Value:  9  Actual Value:  7
Index:  523  Predicted Value:  8  Actual Value:  8
Index:  524  Predicted Value:  8  Actual Value:  8
Index:  525  Predicted Value:  0  Actual Value:  5
Index:  526  Predicted Value:  0  Actual Value:  0
Index:  527  Predicted Value:  6  Actual Value:  1
Index:  528  Predicted Value:  1  Actual Value:  1
Index:  529  Predicted Value:  4  Actual Value:  4
Index:  530  Predicted Value:  7  Actual Value:  8
Index:  531  Predicted Value:  3  Actual Value:  3
Index:  532  Predicted Value:  9  Actual Value:  9
Index:  533  Predicted Value:  0  Actual Value:  0
Index:  534  Predicted Value:  6  Actual Value:  0
Index:  535  Predicted Value:  0  Actual Value:  0
Index:  536  Predicted Value:  

Index:  738  Predicted Value:  3  Actual Value:  3
Index:  739  Predicted Value:  0  Actual Value:  0
Index:  740  Predicted Value:  8  Actual Value:  8
Index:  741  Predicted Value:  1  Actual Value:  7
Index:  742  Predicted Value:  3  Actual Value:  3
Index:  743  Predicted Value:  9  Actual Value:  9
Index:  744  Predicted Value:  6  Actual Value:  6
Index:  745  Predicted Value:  9  Actual Value:  9
Index:  746  Predicted Value:  3  Actual Value:  3
Index:  747  Predicted Value:  5  Actual Value:  5
Index:  748  Predicted Value:  0  Actual Value:  0
Index:  749  Predicted Value:  2  Actual Value:  2
Index:  750  Predicted Value:  7  Actual Value:  7
Index:  751  Predicted Value:  4  Actual Value:  4
Index:  752  Predicted Value:  5  Actual Value:  5
Index:  753  Predicted Value:  1  Actual Value:  1
Index:  754  Predicted Value:  2  Actual Value:  7
Index:  755  Predicted Value:  5  Actual Value:  5
Index:  756  Predicted Value:  8  Actual Value:  8
Index:  757  Predicted Value:  